In [1]:
import sys
import numpy as np
import os
from image_utils import standardize,distance_map_batch,distance_map_batch_v2
from IOU_computations import *
from numpy import newaxis
import matplotlib.pyplot as plt
from dataset_generator_2 import Dataset_sat

import logging
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
from resunet_main_fin import predict,criterion
import torch.backends.cudnn as cudnn
import json

/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
!gpustat

hyperion  Sat May 12 12:03:52 2018
[0] TITAN X (Pascal) | 51'C,   0 % |  5269 / 12189 MB | s132314(5259M)
[1] TITAN X (Pascal) | 55'C,   1 % | 11755 / 12189 MB | maxvo(11745M)
[2] TITAN X (Pascal) | 44'C,   0 % |     0 / 12189 MB |
[3] TITAN X (Pascal) | 37'C,   0 % |     0 / 12189 MB |
[4] TITAN X (Pascal) | 35'C,   0 % |     0 / 12189 MB |
[5] TITAN X (Pascal) | 34'C,   0 % |     0 / 12189 MB |
[6] TITAN X (Pascal) | 23'C,   0 % | 11755 / 12189 MB | maxvo(11745M)
[7] TITAN X (Pascal) | 33'C,   3 % |     0 / 12189 MB |


In [3]:
INPUT_CHANNELS=9
NB_CLASSES=2


###############
DEFAULT_LAYERS=3
DEFAULT_FEATURES_ROOT=32
BATCH_NORM=True

#################
DEFAULT_FILTER_WIDTH=3
DEFAULT_PADDING=1


# DISTANCE_NET='v1'
BINS=15
THRESHOLD=33

# DISTANCE_NET='v2'
# BINS=10
# THRESHOLD=20
# DISTANCE_NET=None

###
DEFAULT_N_RESBLOCKS=1
N_DILATED_CONV=3
DEFAULT_GATED=False
DEFAULT_GROUP_NORM=0

####
DEFAULT_HIDDEN_FEATURES=[32,64,128]
DEFAULT_N_RESBLOCKS=1
N_DILATED_CONV=3
DEFAULT_GATED=False
DEFAULT_GROUP_NORM=0
DEFAULT_PADDING=1

###
DEFAULT_HIDDEN_FEATURES_DILATED=[32,64,128]

WEIGHTS_INIT=False

In [4]:
def generate_dataset(path_validation,validation_batch_size):
    val_generator=Dataset_sat.from_root_folder(path_validation,NB_CLASSES)
    val_loader = DataLoader(val_generator, batch_size=validation_batch_size,shuffle=False, num_workers=1)
    
    return val_loader

In [5]:
def predict_pytorch_val(val_loader,restore_path,prediction_path,file_results,distance_net=None,unet_v=2, model_name=''):
    TMP_IOU=prediction_path+'TMP_IOU/'
    if not os.path.exists(TMP_IOU):
            os.makedirs(TMP_IOU)
    loss_fn=nn.CrossEntropyLoss()
    
    if distance_net=='v1' or distance_net=='v2':
        distance_net_unet=True
    else:
        distance_net_unet=False
    if unet_v==0:
        model=UNet(INPUT_CHANNELS,NB_CLASSES,depth =DEFAULT_LAYERS,n_features_zero =DEFAULT_FEATURES_ROOT,dropout=0,distance_net=distance_net_unet,bins=BINS,batch_norm=BATCH_NORM)
    elif unet_v==1 or unet_v==2 or unet_v==4:
        model=UNet(INPUT_CHANNELS,NB_CLASSES,depth =DEFAULT_LAYERS,n_features_zero =DEFAULT_FEATURES_ROOT,width_kernel=DEFAULT_FILTER_WIDTH,dropout=0,distance_net=distance_net_unet,bins=BINS,batch_norm=BATCH_NORM)
    elif unet_v==3:
        model=UNet(INPUT_CHANNELS,NB_CLASSES,num_hidden_features =DEFAULT_HIDDEN_FEATURES,n_resblocks=DEFAULT_N_RESBLOCKS,num_dilated_convs=N_DILATED_CONV,dropout=0,gated=DEFAULT_GATED,kernel_size=DEFAULT_FILTER_WIDTH,padding=DEFAULT_PADDING, group_norm=DEFAULT_GROUP_NORM)
    elif unet_v==5:
        model=DilatedNetwork( INPUT_CHANNELS,NB_CLASSES,num_hidden_features=DEFAULT_HIDDEN_FEATURES_DILATED,n_resblocks=DEFAULT_N_RESBLOCKS, dropout=0, padding=DEFAULT_PADDING, kernel_size=DEFAULT_FILTER_WIDTH,batch_norm=BATCH_NORM)
    if WEIGHTS_INIT:
        model.apply(weights_init)

    model.cuda()
    cudnn.benchmark = True
    model.load_state_dict(torch.load(restore_path))
    


    loss_v=0
    iou_v=0
    iou_acc_v=0
    f1_v=0

    for i_batch,sample in enumerate(val_loader):
        batch_x=standardize(sample['input'])
        batch_y=sample['groundtruth']

        X = Variable(batch_x.float())
        X=X.permute(0,3,1,2).cuda()  
        Y = Variable(batch_y.float())
        Y=Y.cuda()
        if distance_net=='v1' or distance_net=='v2':
            if distance_net=='v1':
                y_dist=distance_map_batch(batch_y,THRESHOLD,BINS)
            elif distance_net=='v2':
                y_dist=distance_map_batch_2(batch_y,THRESHOLD,BINS)
                
            Y_dist = Variable(y_dist.float())
            Y_dist=Y_dist.cuda()
            probs_dist,probs_seg=predict(model,X,distance_net)
            loss_seg=criterion(Y,probs_seg)
            loss_dist=criterion(Y_dist,probs_dist)
            loss=loss_seg+loss_dist
            prediction_seg_v=probs_seg.data.cpu().numpy()
            groundtruth_seg_v=np.asarray(batch_y)
            prediction_dist_v=probs_dist.data.cpu().numpy()
            groundtruth_dist=np.asarray(y_dist)

        else:
            probs=predict(model,X)
            loss=criterion(Y,probs)
            prediction_seg_v=probs.data.cpu().numpy()
            groundtruth_seg_v=np.asarray(batch_y)
            prediction_dist_v=None
            groundtruth_dist=None

        loss_v+=loss.data[0]


        iou_acc,f1,iou=predict_score_batch(TMP_IOU,np.argmax(groundtruth_seg_v,3),np.argmax(prediction_seg_v,3))
        iou_acc_v+=iou_acc
        iou_v+=iou
        f1_v+=f1

#         if i_batch==random_batch_display:
#             batch_x=np.asarray(batch_x)
#             if batch_x.shape[-1]>3:
#                 pansharp=np.stack((batch_x[:,:,:,5],batch_x[:,:,:,3],batch_x[:,:,:,2]),axis=3)
#             else:
#                 pansharp=batch_x
#             plot_summary(prediction_seg_v,groundtruth_seg_v,prediction_dist_v,groundtruth_dist,pansharp,name,self.prediction_path,save_patches)


    loss_v/=val_loader.__len__()
    iou_v/=val_loader.__len__()  
    iou_acc_v/=val_loader.__len__()  
    f1_v/=val_loader.__len__()  
    logging.info("Model {:s}: Verif loss = {:.4f}, Verification  IOU = {:.4f}, IOU Precision = {:.4f}%, F1 IOU= {:.4f}%".format(model_name,loss_v,iou_v,iou_acc_v,f1_v))

    file_results.write(json.dumps({'name': model_name, 'loss':loss_v, 'IOU':iou_v, 'IOU accuracy':iou_acc_v, 'F1 IOU': f1_v}))
    file_results.flush()
        

In [7]:
batch_size=32

root_folder ='../SPACENET_DATA/SPACENET_DATA_PROCESSED/DATASET/120_x_120_8_bands_pansh/'
validation_loader=generate_dataset(root_folder+'TEST/',batch_size)

prediction_path='TEST_SET_DIST_RESULTS/'
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)
file_results=open(prediction_path+'models_transfer_learning.txt','w')


# ## Dilated meli
# from unet_dilated_meli import DilatedNetwork
# predict_pytorch_val(validation_loader,'MODEL_SPACENET_DILATED_MELI/RESUNET_spacenet_dilated_meliCP84000.pth',prediction_path,file_results,distance_net=None,unet_v=5,model_name='DILATED MELI')

# ## UNET gated dilated
# from unet_val_gated_dilated import UNet
# predict_pytorch_val(validation_loader,'MODEL_SPACENET_DILATE_UNET/RESUNET_spacenet_dilated_unetCP28000.pth',prediction_path,file_results,distance_net=None,unet_v=3,model_name='DILATED UNET BEST LOSS')
# ## UNET gated dilated
# from unet_val_gated_dilated import UNet
# predict_pytorch_val(validation_loader,'MODEL_SPACENET_DILATE_UNET/RESUNET_spacenet_dilated_unetCP284000.pth',prediction_path,file_results,distance_net=None,unet_v=3,model_name='DILATED UNET BEST IOU METRICS')
## UNET NO dist
from unet_val_2 import UNet
DISTANCE_NET=None
predict_pytorch_val(validation_loader,'MODEL_SPACENET_NODIST5/RESUNET_spacenet_nodist_5CP60000.pth',prediction_path,file_results,distance_net=DISTANCE_NET,unet_v=2,model_name='RESUNET NO DIST')
## UNET Dist
from unet_val_2 import UNet
DISTANCE_NET='v1'
predict_pytorch_val(validation_loader,'MODEL_SPACENET_DIST5/RESUNET_spacenet_dist_5CP84000.pth',prediction_path,file_results,distance_net=DISTANCE_NET,unet_v=2,model_name='RESUNET DIST')
# ## UNET Dist
# from unet_val_2 import UNet
# predict_pytorch_val(validation_loader,'MODEL_SPACENET_DIST5/RESUNET_spacenet_dist_5CP256000.pth',prediction_path,file_results,distance_net=True,unet_v=2,model_name='RESUNET DIST BEST IOU METRICS')


# batch_size=32
# root_folder = '../2_DATA_GHANA/DATASET/120_x_120_8_pansh/'
# # root_folder ='../SPACENET_DATA/SPACENET_DATA_PROCESSED/DATASET/120_x_120_8_bands_pansh/'
# validation_loader=generate_dataset(root_folder+'TEST/',batch_size)

# prediction_path='TEST_SET_GHANA_RESULTS/'
# if not os.path.exists(prediction_path):
#     os.makedirs(prediction_path)
# file_results=open(prediction_path+'models_transfer_learning.txt','w')


# ## UNET NO DIST SCRATCH
# from unet_val_2 import UNet
# DISTANCE_NET=None

# predict_pytorch_val(validation_loader,'MODEL_GHANA_NODIST_NODATA_AUG/RESUNET_ghana_nodata_augCP38000.pth',prediction_path,file_results,distance_net=DISTANCE_NET,unet_v=2,model_name='RESUNET SCRATCH NO DIST 38000 ')
# ## UNET NO DIST tf best loss 
# from unet_val_2 import UNet
# DISTANCE_NET=None
# predict_pytorch_val(validation_loader,'MODEL_GHANA_TF_NO_AUG_NODIST5/RESUNET_ghana_tf_no_aug_nodist5CP12000.pth',prediction_path,file_results,distance_net=DISTANCE_NET,unet_v=2,model_name='RESUNET NO DIST TF BEST LOSS 12000')
# ## UNET NO DIST tf best metrics 
# from unet_val_2 import UNet
# DISTANCE_NET=None
# predict_pytorch_val(validation_loader,'MODEL_GHANA_TF_NO_AUG_NODIST5/RESUNET_ghana_tf_no_aug_nodist5CP46000.pth',prediction_path,file_results,distance_net=DISTANCE_NET,unet_v=2,model_name='RESUNET NO DIST TF IOU METRICS 46000')

# ## UNET DIST scratch
# from unet_val_2 import UNet

# DISTANCE_NET='v1'

# predict_pytorch_val(validation_loader,'MODEL_GHANA_DIST_NODATA_AUG/RESUNET_ghana_dist_nodata_augCP44000.pth',prediction_path,file_results,distance_net=DISTANCE_NET,unet_v=2,model_name='RESUNET DIST SCRATCH 44000')
# ## UNET DIST TF
# from unet_val_2 import UNet
# DISTANCE_NET='v1'

# predict_pytorch_val(validation_loader,'MODEL_GHANA_TF_NO_AUG_DIST5_560000/RESUNET_ghana_tf_no_aug_nodist5_56000CP24000.pth',prediction_path,file_results,distance_net=DISTANCE_NET,unet_v=2,model_name='RESUNET DIST TF 24000  ')


2018-05-12 12:17:05,807 Model RESUNET NO DIST: Verif loss = 0.1738, Verification  IOU = 0.7058, IOU Precision = 78.0288%, F1 IOU= 68.6958%
2018-05-12 12:31:38,295 Model RESUNET DIST: Verif loss = 0.6054, Verification  IOU = 0.7119, IOU Precision = 78.9482%, F1 IOU= 67.8948%
